# Graph NN for MMWave Sensor filtering
The idea is to train a classifier to distinguish between fake points and actual ones

In [3]:
'''
    Requirements:
     * TF:      2.7.0
     * Keras:   2.7.0
'''

import tensorflow as tf
import keras
from keras import layers

'''
    Setting Up matplotlib for paper compliant figures
    (this should avoid problems when compiling latex stuff)
'''
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers

import optuna
import numpy as np
from scipy import stats
import json
import os

# Where datasets are stored
run_path = "data/"
experiments_path = "experiments/"

def load_datasets(init_path):
    '''
        Return dataset as list of pointclouds
        ---
        Parameters:
        * init_path: string; path to folder holding folders of datasets
                     (expected folder structure: 'init_path/run_X/labelled_mmw_run_X.json')
    '''
    data = []
    for run in sorted(os.listdir(init_path)):
        if "run_" in run:
            data.extend(json.load(open(init_path+run+"/labelled_mmw_"+run+".json"))['labelled_mmw'])
    return data

def load_train_test_data(init_path, train_filter=[], test_filter=[]):
    '''
        Return dataset as list of pointclouds
        ---
        Parameters:
        * init_path: string; path to folder holding folders of datasets
                     (expected folder structure: 'init_path/run_X/labelled_mmw_run_X.json')
    '''
    train = []
    test = []
    for run in sorted(os.listdir(init_path)):
        if "run_" in run:
            if (int(run.split('_')[-1]) in train_filter):
                train.extend(json.load(open(init_path+run+"/labelled_mmw_"+run+".json"))['labelled_mmw'])
            elif (int(run.split('_')[-1]) in test_filter):
                test.extend(json.load(open(init_path+run+"/labelled_mmw_"+run+".json"))['labelled_mmw'])
    return train, test

def get_data_and_label(data, points_per_cloud=200):
    '''
        Return samples for training as np.array, divided as unlabelled data and related labels.
        ---
        Parameters:
        * data: list of point clouds. (Usually loaded with function load_datasets)
        * points_per_cloud: number of points to be found in each point cloud. (Default is 200)
    '''
    d_x, d_y = [], []
    for pc in data:
        for i in range(0, len(pc), points_per_cloud): 
            if len(pc[i:i+points_per_cloud]) == points_per_cloud:
                t_ = np.array(pc[i:i+points_per_cloud])[:, :3]
                d_x.append(t_)
                d_y.append(np.array(pc[i:i+points_per_cloud], dtype=np.float32)[:, -1])
                # d_y.append(tf.one_hot(np.array(pc[i:i+points_per_cloud], dtype=np.float32)[:, -1], 2)) # One Hotted
    d_x, d_y = np.stack(d_x), np.stack(d_y)
    return d_x, d_y

train_filter=[4,6,8,9,10,11,12,50,53,54,56,58,59,61,62,63,65,66,67,69]
test_filter=[3,7,49,51,55,57,64,68,70]

# dataset = load_datasets(run_path)
train, test = load_train_test_data(run_path, train_filter=train_filter, test_filter=test_filter)

# Separate Train and Test data
# d_len = int(len(dataset)*0.7)
# train, test = dataset[:d_len], dataset[d_len:]


# Shuffle point cloud dataset
np.random.shuffle(train)
np.random.shuffle(test)

# Get X and Y data for training
train_x, train_y = get_data_and_label(train)
test_x, test_y = get_data_and_label(test)

# Dynamic Graph CNN 
#### https://github.com/WangYueFt/dgcnn
#### https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow

In [4]:
def gnn_conv2d(inputs,
            filters,
            kernel_size,
            stride=[1, 1],
            padding='SAME',
            use_xavier=True,
            stddev=1e-3,
            activation_fn=tf.nn.elu,
            bn=False):

    x = layers.Conv2D(
        filters, 
        kernel_size, 
        strides=stride, 
        padding=padding,
        activation=activation_fn,
        kernel_initializer='glorot_uniform' if use_xavier else keras.initializers.TruncatedNormal(stddev=stddev),
        bias_initializer='zeros'
    )(inputs)

    if bn: x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    return x

def gnn_dense(inputs,
            units,
            use_xavier=True,
            stddev=1e-3,
            activation_fn=tf.nn.elu,
            bn=False):
            
    x = layers.Dense(units,
        activation=activation_fn,
        kernel_initializer='glorot_uniform' if use_xavier else keras.initializers.TruncatedNormal(stddev=stddev),
        bias_initializer='zeros'
    )(inputs)

    if bn: x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    return x

def lambda_get_adj_matr(input):
    pcT = layers.Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(input)
    pc_inn = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))( (input, pcT) )
    pc2 = layers.Lambda(lambda x: tf.reduce_sum(tf.square(x), axis=-1, keepdims=True))(input)
    pc2T = layers.Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(pc2)
    output = layers.Lambda(lambda x: x[0] + -2 * x[1] + x[2])( (pc2, pc_inn, pc2T) )
    # Uncomment line below to use reciprocal of adj matrix (1/distance)
    # output = layers.Lambda(lambda x: tf.math.reciprocal(x))(output)
    return output

def lambda_knn(adj, k=20):
    x = layers.Lambda(lambda x: tf.math.top_k(-x[0], x[1]))( (adj, k) )
    return x.indices

def lambda_edge_feature(inputs, nn_idxs, k=20, num_points=200, num_dims=3):

    pc_central = inputs
    batch_size = tf.shape(inputs)[0]

    idx_ = layers.Lambda(lambda x: tf.range(x[0]) * x[1])( (batch_size, num_points) )
    idx_ = layers.Lambda(lambda x: tf.reshape(x[0], (x[1], 1, 1)))( (idx_, batch_size) )
    # Adding to list of idxs of k points the points themselves
    pc_temp1 = layers.Lambda(lambda x: x[0]+x[1])( (nn_idxs, idx_) )

    # Flattening of points into a list of coordinates (x,y,z)
    pc_flat = layers.Lambda(lambda x: tf.reshape(x[0], [-1, x[1]]))( (inputs, num_dims) )

    # Collect points from computed idxs
    pc_neighbors = layers.Lambda(lambda x: tf.gather(x[0], x[1]) )( (pc_flat, pc_temp1) )

    # Reshape points into shape (batch, num_points, NEW_AXIS = 1, num_dims)
    pc_central = layers.Lambda(lambda x: tf.expand_dims(x, axis=-2))(pc_central)
    # Points are repeated k-times along new dimension ==> (batch, num_points, k, num_dims)
    pc_central = layers.Lambda(lambda x: tf.tile(x[0], [1, 1, x[1], 1]))( (pc_central, k) )

    pc_temp2 = layers.Lambda(lambda x: tf.subtract(x[0], x[1]))( (pc_neighbors, pc_central) )
    edge_feature = layers.Lambda(lambda x: tf.concat((x[0], x[1]), axis=-1))((pc_central, pc_temp2))
    return edge_feature

def gnn_tnet(inputs, num_dims, tnet_shapes, bn=False):
    batch_size = tf.shape(inputs)[0]
    for filt in tnet_shapes[0]:
        x = gnn_conv2d(inputs, filters=filt, kernel_size=[1,1], bn=bn)
    x = tf.reduce_max(x, axis=-2, keepdims=True)
    for filt in tnet_shapes[1]:
        x = gnn_conv2d(inputs, filters=filt, kernel_size=[1,1], bn=bn)
    x = layers.GlobalMaxPooling2D(keepdims=True)(x)
    x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], y[2])))( [x, batch_size, x.shape[-1]] )

    for neur in tnet_shapes[2]:
        x = gnn_dense(x, neur, bn)
    
    bias = keras.initializers.Constant(np.eye(num_dims).flatten())
    x = layers.Dense(
        num_dims * num_dims,
        kernel_initializer="zeros",
        bias_initializer=bias,
    )(x)
    feat_T = layers.Reshape((num_dims, num_dims))(x)
    return feat_T

####################################################################################################################

# test_name = "test_0"

In [7]:
# function to test custom losses
def custom_loss(pred, labels):
    # loss = tf.compat.v1.losses.softmax_cross_entropy(onehot_labels=labels, logits=pred, label_smoothing=0.2)
    # classify_loss = tf.reduce_mean(loss)
    loss = tf.reduce_mean(tf.reduce_sum(tf.math.square(tf.math.subtract(labels, pred)), axis=-1))
    return loss

# Callback to save good models. Threshold is on validation accuracy.
class ValAccThresh_CB(keras.callbacks.Callback):
    def __init__(self, thresh=0.85, experiments_path="experiments/", test_name="test"):
        self.thresh = thresh
        super(keras.callbacks.Callback, self).__init__()
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
        self.stopped_epoch = 0
        self.experiments_path = experiments_path
        self.test_name = test_name

    def on_epoch_end(self, epoch, logs=None):
        self.current_epoch += 1
        val_key = ""
        for k in logs.keys():
            if "val" in k and "accuracy" in k:
                val_key = k
                break
        if val_key == "": print(" Validation Accuracy key not found.")

        current = logs.get(val_key)
        # current = logs.get("val_sparse_categorical_accuracy")
        # current = logs.get("val_accuracy")
        if current >= self.thresh:
            self.thresh = current
            self.model.save_weights(self.experiments_path+self.test_name+"/best_weights/cp-"+str(epoch)+".ckpt")
            print(" New good model saved.")

# Callback to save history for post-processing
# filename=experiments_path+test_name+"/history.csv"
# history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [8]:
def build_model(inputs, 
        num_points, num_dims, k,
        tnet_shape,
        conv_gnns,
        dense_gnn,
        classes=2):
    '''
        Returns the outputs of the model to be compiled.
        ---
        Arguments: 
        * inputs:       Expected (None, 3). instance of tf.Input.
        * num_points:   Number of points per point cloud. Default is 200
        * num_dims:     Number of dimensions per point. Default is 3 (x, y, z)
        * k:            K nearest neighbors
        * tnet_shape:   Array of three lists. (each list's length is the number of layers for that section)
                        1st is a list of filters for convolutional layers before reduce_max.
                        2nd is a list of filters for convolutional layers after reduce_max.
                        3rd is a list of neurons for dense layers after max pooling.
        * conv_gnns:    list. Each row is composed of two lists.
                        1st is a list of filters for convolutional layers before computing edge features.
                        2nd is a list of filters for convolutional layers after computing edge features.
        * dense_gnn:    list of neuorns for dense layers at the end of the network.
        * classes:      number of classes to classify.
    '''

    adj = lambda_get_adj_matr(inputs)
    nn_idxs = lambda_knn(adj, k)
    edge_feats = lambda_edge_feature(inputs, nn_idxs, k, num_points, num_dims)
    feat_T = gnn_tnet(edge_feats, num_dims, tnet_shape, bn=True)
    pc_tf = layers.Dot(axes=(-1, -2))([inputs, feat_T]) # Apply affine transformation to input features

    adj = lambda_get_adj_matr(pc_tf)
    nn_idxs = lambda_knn(adj, k)
    edge_feats = lambda_edge_feature(pc_tf, nn_idxs, k, num_points, num_dims)

    for l in conv_gnns:
        x = edge_feats
        for gc_filt in l[0]:
            x = gnn_conv2d(x, gc_filt, [1,1], bn=True)
        x = tf.reduce_max(x, axis=-2, keepdims=True)
        x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], num_points, l[0][-1])))( [x, tf.shape(inputs)[0]] )

        adj = lambda_get_adj_matr(x)
        nn_idxs = lambda_knn(adj, k)
        edge_feats = lambda_edge_feature(x, nn_idxs, k, num_points, l[0][-1])
        x = edge_feats
        for gc_filt in l[1]:
            x = gnn_conv2d(x, gc_filt, [1,1], bn=True)
        x = tf.reduce_max(x, axis=-2, keepdims=True)
        x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], num_points, l[1][-1])))( [x, tf.shape(inputs)[0]] )

    for w_ in dense_gnn:
        x = gnn_dense(x, w_, bn=True)

    outputs = layers.Dense(classes, activation="softmax")(x)
    return outputs

def objective(trial):
    test_name = "fixed_test_"+str(trial.number)
    filename=experiments_path+test_name+"/history.csv"
    history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
    ############################ HyperParameter Setup ############################
    ######################### Check build_model for docs #########################
    k = trial.suggest_int('k', 5,75) #30
    batch_size = trial.suggest_int('batch_size', 8,128) #16
    # tnet_before_max = trial.suggest_int('tnet_before_max', 1,3)
    # tnet_before = []
    # for i in range(tnet_before_max):
    #     tnet_before.append(trial.suggest_int('tnet_beforemax_layer_'+str(i), 8,128))
    # tnet_after = []
    # tnet_after_max = trial.suggest_int('tnet_after_max', 1,3)
    # for i in range(tnet_after_max):
    #     tnet_after.append(trial.suggest_int('tnet_aftermax_layer_'+str(i), 8,128))
    # tnet_dense = []
    # tnet_dense_layers = trial.suggest_int('tnet_dense_layers', 1,3)
    # for i in range(tnet_dense_layers):
    #     tnet_dense.append(trial.suggest_int('tnet_dense_layer_'+str(i), 16,256))
    tnet_before = [32]
    tnet_after = [128]
    tnet_dense = [128, 128]
    tnet_shape = [tnet_before, tnet_after, tnet_dense]

    # gc_layers = trial.suggest_int('gc_layers', 1,3) #1
    conv_gnns = []
    # for _ in range(gc_layers):
    #     before_edge_gcl = trial.suggest_int('before_edge_gcl', 1,3) #2
    #     after_edge_gcl = trial.suggest_int('after_edge_gcl', 1,3) #2

    #     bfr_edge = []
    #     for i in range(before_edge_gcl):
    #         bfr_edge.append(trial.suggest_int('before_edge_gcl_'+str(i), 8,128))
            
    #     aft_edge = []
    #     for i in range(after_edge_gcl):
    #         aft_edge.append(trial.suggest_int('after_edge_gcl_'+str(i), 8,128))
            
    #     conv_gnns.append([bfr_edge, aft_edge])
    bfr_edge = [32, 64]
    aft_edge = [128]
    conv_gnns.append([bfr_edge, aft_edge])
    # dense_layers = trial.suggest_int('dense_layers', 1,3)
    # dense_gnn = []
    # for i in range(dense_layers):
    #     dense_gnn.append(trial.suggest_int('dense_layer_'+str(i), 16,256))
    dense_gnn = [256, 128]
        
    lr = trial.suggest_float('lr', 0.0001, 0.1)
    steps_per_epoch=trial.suggest_int('steps_per_epoch', 15,100)

    validation_steps=25     # Static
    num_points = 200        # Static
    num_dims = 3            # Static
    ##############################################################################

    inputs = keras.Input(shape=(None, num_dims))
    
    outputs = build_model(inputs,
                    num_points, num_dims, k,
                    tnet_shape,
                    conv_gnns,
                    dense_gnn
                )
    model = keras.Model(inputs=[inputs], outputs=outputs, name="gnn_pointnet")

    opt_pi = tf.optimizers.Adam(learning_rate =  lr )
    # opt_pi = tf.optimizers.RMSprop(learning_rate =  lr )
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=opt_pi, metrics=[keras.metrics.SparseCategoricalAccuracy()])
    # model.compile(loss=tf.nn.sparse_softmax_cross_entropy_with_logits , optimizer=opt_pi, metrics=['accuracy'])
    # model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=opt_pi, metrics=['accuracy'])
    # model.compile(loss=custom_loss, optimizer=opt_pi, metrics=['accuracy'])

    checkpoint_path = experiments_path+test_name+"/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, 
        verbose=0, 
        save_weights_only=False,
        save_freq=10*batch_size)
        
    latest = tf.train.latest_checkpoint(experiments_path+test_name+"/")
    if latest:
        model.load_weights(latest)
        latest_ep = int(latest.split('/')[-1].split('-')[-1].split('.')[0])
        print(" Model loaded correctly:", latest, " - Epoch ", latest_ep)
    else:
        print(" The model could not be loaded properly: ", latest)
        model.save(checkpoint_path.format(epoch=0))
        latest_ep = 0

    # Use CPU as default due to GPU's memory issues
    with tf.device('/CPU:0'):
        history = model.fit(
            train_x, 
            train_y, 
            
            initial_epoch=latest_ep,
            batch_size=batch_size, 
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,

            validation_split=0.3,
            epochs=150 - latest_ep, # Train for 150 epochs to find the configuration that can later be trained for more epochs.
            shuffle=True,
            callbacks=[ValAccThresh_CB(thresh=0.9), cp_callback, history_logger],
            use_multiprocessing=False,
            workers=8,
        )
    return np.mean(history.history['val_sparse_categorical_accuracy'][-10:])

In [6]:
# Actually train
storage = optuna.storages.RDBStorage(url="sqlite:///gnn_fixed.db", engine_kwargs={"connect_args": {"timeout": 5}})
study = optuna.create_study(study_name="gnn_denoising_fixed", storage=storage, load_if_exists=True, direction="maximize")
study.optimize(objective, n_trials=1000)

[I 2023-01-18 11:58:17,322] Using an existing study with name 'gnn_denoising_fixed' instead of creating a new one.
2023-01-18 11:58:18.201583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 11:58:18.919930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 11:58:18.920117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 11:58:18.957020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-

 The model could not be loaded properly:  None


2023-01-18 11:58:24.080988: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 45849600 exceeds 10% of free system memory.


Epoch 1/150


2023-01-18 11:58:27.379891: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 499200000 exceeds 10% of free system memory.
2023-01-18 11:58:27.604712: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 499200000 exceeds 10% of free system memory.
2023-01-18 11:58:27.923571: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 499200000 exceeds 10% of free system memory.
2023-01-18 11:58:28.251407: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 124800000 exceeds 10% of free system memory.


...

 The model could not be loaded properly:  None
Epoch 1/150
25/25 [==============================] - 81s 3s/step - loss: 0.8640 - sparse_categorical_accuracy: 0.6092 - val_loss: 0.6820 - val_sparse_categorical_accuracy: 0.5347
Epoch 2/150
25/25 [==============================] - 79s 3s/step - loss: 0.6326 - sparse_categorical_accuracy: 0.6811 - val_loss: 0.6526 - val_sparse_categorical_accuracy: 0.6724
Epoch 3/150
25/25 [==============================] - 79s 3s/step - loss: 0.6425 - sparse_categorical_accuracy: 0.6626 - val_loss: 0.7834 - val_sparse_categorical_accuracy: 0.3450
Epoch 4/150
25/25 [==============================] - 79s 3s/step - loss: 0.6297 - sparse_categorical_accuracy: 0.6771 - val_loss: 0.6561 - val_sparse_categorical_accuracy: 0.6724
Epoch 5/150
25/25 [==============================] - 80s 3s/step - loss: 0.6274 - sparse_categorical_accuracy: 0.6743 - val_loss: 0.6782 - val_sparse_categorical_accuracy: 0.6464
Epoch 6/150
25/25 [==============================] - 79s 3

[I 2023-01-26 10:21:46,328] Trial 93 finished with value: 0.6358176112174988 and parameters: {'k': 27, 'batch_size': 67, 'lr': 0.007398913177574749, 'steps_per_epoch': 25}. Best is trial 84 with value: 0.6879163682460785.


 The model could not be loaded properly:  None
Epoch 1/150
30/30 [==============================] - 89s 3s/step - loss: 0.8393 - sparse_categorical_accuracy: 0.5930 - val_loss: 0.6638 - val_sparse_categorical_accuracy: 0.6707
Epoch 2/150
30/30 [==============================] - 87s 3s/step - loss: 0.6535 - sparse_categorical_accuracy: 0.6573 - val_loss: 0.6591 - val_sparse_categorical_accuracy: 0.6719
Epoch 3/150
30/30 [==============================] - 87s 3s/step - loss: 0.6280 - sparse_categorical_accuracy: 0.6851 - val_loss: 0.6355 - val_sparse_categorical_accuracy: 0.6719
Epoch 4/150
30/30 [==============================] - 87s 3s/step - loss: 0.6365 - sparse_categorical_accuracy: 0.6669 - val_loss: 0.6329 - val_sparse_categorical_accuracy: 0.6719
Epoch 5/150
30/30 [==============================] - 87s 3s/step - loss: 0.6313 - sparse_categorical_accuracy: 0.6722 - val_loss: 0.6418 - val_sparse_categorical_accuracy: 0.6719
Epoch 6/150
30/30 [==============================] - 88s 3

[I 2023-01-26 14:01:29,784] Trial 94 finished with value: 0.6844311416149139 and parameters: {'k': 28, 'batch_size': 61, 'lr': 0.005139378695279151, 'steps_per_epoch': 30}. Best is trial 84 with value: 0.6879163682460785.


 The model could not be loaded properly:  None
Epoch 1/150
30/30 [==============================] - 80s 3s/step - loss: 0.8234 - sparse_categorical_accuracy: 0.6002 - val_loss: 0.6297 - val_sparse_categorical_accuracy: 0.6732
Epoch 2/150
30/30 [==============================] - 77s 3s/step - loss: 0.6569 - sparse_categorical_accuracy: 0.6651 - val_loss: 0.6357 - val_sparse_categorical_accuracy: 0.6732
Epoch 3/150
30/30 [==============================] - 77s 3s/step - loss: 0.6348 - sparse_categorical_accuracy: 0.6812 - val_loss: 0.6358 - val_sparse_categorical_accuracy: 0.6732
Epoch 4/150
30/30 [==============================] - 77s 3s/step - loss: 0.6330 - sparse_categorical_accuracy: 0.6746 - val_loss: 0.6511 - val_sparse_categorical_accuracy: 0.6732
Epoch 5/150
30/30 [==============================] - 78s 3s/step - loss: 0.6338 - sparse_categorical_accuracy: 0.6691 - val_loss: 0.6285 - val_sparse_categorical_accuracy: 0.6732
Epoch 6/150
30/30 [==============================] - 77s 3

## Testing Configurations Manually

In [ ]:
test_name = "manual_test_01"

k=27
batch_size=66
steps_per_epoch=33
validation_steps=25
lr = 0.06283

num_points=200
num_dims=3

checkpoint_path = experiments_path+test_name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=False,
    save_freq=10*steps_per_epoch)

inputs = keras.Input(shape=(None, 3))

tnet_shape = [[32], [128], [128,128]]
conv_gnns = [[[32,64], [128]]]
dense_gnn = [256,128]

outputs = build_model(inputs,
                    num_points, num_dims, k,
                    tnet_shape,
                    conv_gnns,
                    dense_gnn
                )

model = keras.Model(inputs=[inputs], outputs=outputs, name=test_name+"net")


opt_pi = tf.optimizers.Adam(learning_rate=lr)
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=opt_pi, metrics=[keras.metrics.SparseCategoricalAccuracy()])

# Try to load the model. If it does not exist, create it.
latest = tf.train.latest_checkpoint(experiments_path+test_name+"/")
if latest:
    # https://www.tensorflow.org/tutorials/keras/save_and_load
    # model.load(latest)
    model = tf.keras.models.load_model(latest)
    latest_ep = int(latest.split('/')[-1].split('-')[-1].split('.')[0])
    print(" Model loaded correctly:", latest, " - Epoch ", latest_ep)
else:
    print(" The model could not be loaded properly: ", latest)
    model.save(checkpoint_path.format(epoch=0))
    latest_ep = 0

# This grants no overwriting of the history file
filename=experiments_path+test_name+"/history"+str(latest_ep)+".csv"
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

# Use CPU as default due to GPU's memory issues
with tf.device('/CPU:0'):
    history = model.fit(
        train_x, 
        train_y, 
        
        initial_epoch=latest_ep,
        batch_size=batch_size, 
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,

        validation_split=0.3,
        epochs=20000,
        shuffle=True,
        callbacks=[ValAccThresh_CB(thresh=0.85, experiments_path=experiments_path, test_name=test_name), cp_callback, history_logger],
        use_multiprocessing=False,
        workers=8,
    )

